<a href="https://colab.research.google.com/github/abbi163/Coursera_Capstone/blob/master/Data_Analytics_with_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
## Task 1: Install Pyspark !!

!pip install pyspark

     |████████████████████████████████| 204.7MB 56kB/s 
     |████████████████████████████████| 204kB 30.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=e48cb744646be71003f20d8753bd3aa2fda24ba18cb58ec5967f52d5e8cacff3
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc, col, max, struct
import matplotlib.pyplot as plt


In [17]:
## Creating sparksession 
spark =  SparkSession.builder.appName('spark_app').getOrCreate()

## Task 2: 
### Importing the Listning.csv file !

In [18]:
listning_csv_path = '/content/drive/My Drive/dataset/listenings.csv'

# We use spark method called read which read csv format data. 
# 'inferSchema' = True, =>, infers columns types based on the data type we have
# 'header' = Ture => same column name as in csv file. 
listning_df = spark.read.format('csv')\
                        .option('inferSchema', True)\
                        .option('header', True)\
                        .load(listning_csv_path)

# listning_df is a pyspark dataframe. 

In [19]:
# checking what is inside listning_df using show method
listning_df.show(10)

+-----------+-------------+--------------------+------------+--------------------+
|    user_id|         date|               track|      artist|               album|
+-----------+-------------+--------------------+------------+--------------------+
|000Silenced|1299680100000|           Price Tag|    Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|    Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|       Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|       Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|   The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|    MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|     Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|   Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      Gojira|The Way of All Flesh|
|000

In [20]:
# dropping useless columns
listning_df = listning_df.drop('date')
# dropping the null rows
listning_df = listning_df.na.drop()
# checking the dataset again
listning_df.show(5) 

+-----------+--------------------+---------+-----------------+
|    user_id|               track|   artist|            album|
+-----------+--------------------+---------+-----------------+
|000Silenced|           Price Tag| Jessie J|      Who You Are|
|000Silenced|Price Tag (Acoust...| Jessie J|        Price Tag|
|000Silenced|Be Mine! (Ballad ...|    Robyn|         Be Mine!|
|000Silenced|            Acapella|    Kelis|         Acapella|
|000Silenced|   I'm Not Invisible|The Tease|I'm Not Invisible|
+-----------+--------------------+---------+-----------------+
only showing top 5 rows



In [21]:
# let's see the schema using method called printSchema
listning_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- track: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- album: string (nullable = true)



In [24]:
# let's see the shape of our dataframe!
# check number of rows and columns in the dataset
shape = (listning_df.count(), len(listning_df.columns))
print(shape)
# Pyspark dataframe doesnot have shape property like pandas dataframe

(13758905, 4)


In [25]:
## Apply some query to extract useful information !!

# Query 0: Select 2 columns, track and artist
# returns dataframe which is stored in Q0 variable
q0 = listning_df.select('artist', 'track') 
q0.show(5)

+---------+--------------------+
|   artist|               track|
+---------+--------------------+
| Jessie J|           Price Tag|
| Jessie J|Price Tag (Acoust...|
|    Robyn|Be Mine! (Ballad ...|
|    Kelis|            Acapella|
|The Tease|   I'm Not Invisible|
+---------+--------------------+
only showing top 5 rows



In [27]:
#Query 1: Let's find all the records of users who have listened to Rihanna
q1 = listning_df.select('*').filter(listning_df.artist == 'Rihanna')
q1.show(10)

+-----------+--------------------+-------+--------------------+
|    user_id|               track| artist|               album|
+-----------+--------------------+-------+--------------------+
|000Silenced|Don't Stop The Mu...|Rihanna|Addicted 2 Bassli...|
|000Silenced|           Disturbia|Rihanna|Good Girl Gone Ba...|
|00williamsl|   Hatin On The Club|Rihanna|              Random|
|00williamsl|   Hatin On The Club|Rihanna|              Random|
|00williamsl|         Complicated|Rihanna|                Loud|
|00williamsl|What's My Name (f...|Rihanna|                Loud|
|00williamsl|Kanye West feat R...|Rihanna|                Loud|
|    0502008|Only Girl (In the...|Rihanna|                Loud|
|      0rdos|Pon De Replay (Re...|Rihanna|    Music of the Sun|
|      0rdos|          Now I Know|Rihanna|    Music of the Sun|
+-----------+--------------------+-------+--------------------+
only showing top 10 rows



In [33]:
# Query 2: Find top 10 users who is fan of Rihanna !

# Step 1:
#  Select all the users who have listened to Rihanna

# Step 2:
# Perform agregation to count how many time each users have listened to rihanna

q2 = listning_df.select('user_id')\
                .filter(listning_df.artist == 'Rihanna')\
                .groupby('user_id')\
                .agg(count('user_id').alias('count'))\
                .orderBy(desc('count'))\
                .limit(10)

# agg -> aggregation, .alias('count') -> we are giving new column name, count !
# orderBy is used for sorting either in ascending or descending order 
# as we need top 10 fans, we will use limit method for finding top 10 columns

q2.show()

+---------------+-----+
|        user_id|count|
+---------------+-----+
|        thiessu|  179|
|    eyessetkyle|  166|
|           adxx|  164|
|missnumberthree|  156|
|helloiamnatalie|  128|
|          nmjnb|  124|
|        AndyyyA|  123|
|     BIGBANG186|  121|
|     mixedvibes|  120|
|       AndyKitt|  115|
+---------------+-----+



In [37]:
# Query 3: Find top 10 famours track!

# Note: There can by 2 same track name of 2 different artists
# So select artist and track and use groupby on both, rather than just track
# if more than one column, in count use '*' instead of both names!
q3 = listning_df.select('artist','track')\
                .groupby('artist', 'track')\
                .agg(count('*').alias('count'))\
                .orderBy(desc('count'))\
                .limit(10)
q3.show()

+--------------+-----------------+-----+
|        artist|            track|count|
+--------------+-----------------+-----+
| Justin Bieber|            Sorry| 3381|
|Arctic Monkeys| Do I Wanna Know?| 2865|
|      Bon Iver|      Skinny Love| 2836|
|          Zayn|       PILLOWTALK| 2701|
|   The Killers|    Mr Brightside| 2690|
|       Rihanna|             Work| 2646|
|      Bastille|          Pompeii| 2606|
|Mumford & Sons|  Little Lion Man| 2520|
|Mumford & Sons|         The Cave| 2485|
| Justin Bieber|What Do You Mean?| 2481|
+--------------+-----------------+-----+



In [40]:
# Query 4, Famous top 10 tracks of Rihanna !!

q4 = listning_df.filter(listning_df.artist == 'Rihanna')\
                .select('artist', 'track')\
                .groupby('artist', 'track')\
                .agg(count('*').alias('count'))\
                .orderBy(desc('count'))\
                .limit(10)
q4.show()

+-------+--------------------+-----+
| artist|               track|count|
+-------+--------------------+-----+
|Rihanna|                Work| 2646|
|Rihanna|Only Girl (In the...| 1749|
|Rihanna|We Found Love (fe...| 1575|
|Rihanna|                 S&M| 1307|
|Rihanna|            Rude Boy| 1303|
|Rihanna|            Diamonds| 1224|
|Rihanna|      Kiss it Better|  945|
|Rihanna| Where Have You Been|  844|
|Rihanna|Cheers (Drink to ...|  697|
|Rihanna|           Needed Me|  679|
+-------+--------------------+-----+

